## Allocating Elastic IP address

In [1]:
import boto3

AWS_REGION = "us-east-1"
EC2_CLIENT = boto3.client('ec2', region_name=AWS_REGION)

ElasticIP = EC2_CLIENT.allocate_address( Domain='vpc',
                                    NetworkBorderGroup=AWS_REGION,
                                    TagSpecifications=[{
                                            'ResourceType': 'elastic-ip',
                                            'Tags': [{
                                                        'Key': 'Name',
                                                        'Value': 'Lab Elastic IP'
                                                    }]
                                        },
                                    ])

ElasticIP.keys()

dict_keys(['PublicIp', 'AllocationId', 'PublicIpv4Pool', 'NetworkBorderGroup', 'Domain', 'ResponseMetadata'])

In [2]:
ElasticIP['AllocationId']

'eipalloc-06730daea269bd751'

## Listing and describing Elastic IP addresses

### with filter

In [3]:
response = EC2_CLIENT.describe_addresses(
                                        Filters=[{
                                                'Name': 'tag:Name',
                                                'Values': ['Lab Elastic IP']
                                            }]
                                        )
for address in response['Addresses']:
    print(address)

{'PublicIp': '35.173.46.64', 'AllocationId': 'eipalloc-06730daea269bd751', 'Domain': 'vpc', 'Tags': [{'Key': 'Name', 'Value': 'Lab Elastic IP'}], 'PublicIpv4Pool': 'amazon', 'NetworkBorderGroup': 'us-east-1'}


### without filter

In [4]:
response = EC2_CLIENT.describe_addresses()
for address in response['Addresses']:
    print(address)


{'PublicIp': '35.173.46.64', 'AllocationId': 'eipalloc-06730daea269bd751', 'Domain': 'vpc', 'Tags': [{'Key': 'Name', 'Value': 'Lab Elastic IP'}], 'PublicIpv4Pool': 'amazon', 'NetworkBorderGroup': 'us-east-1'}


## Attaching an Elastic IP to an EC2 Instance

In [6]:
EC2_RESOURCE = boto3.resource('ec2', region_name=AWS_REGION)

instances = EC2_RESOURCE.instances.filter(
    Filters=[
        {
            'Name': 'tag:Name',
            'Values': ['my-ec2-instance']
        }
    ]
)
for instance in instances:
    print(f'  - Instance ID: {instance.id}')

  - Instance ID: i-048f0f8d64736cb28


In [7]:
response = EC2_CLIENT.associate_address(InstanceId='i-048f0f8d64736cb28',
                                        AllocationId=ElasticIP['AllocationId'])
response

{'AssociationId': 'eipassoc-05457dbd039ff0017',
 'ResponseMetadata': {'RequestId': '02d091fd-5149-41ce-ad36-7faab0dc84aa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '02d091fd-5149-41ce-ad36-7faab0dc84aa',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '295',
   'date': 'Fri, 04 Feb 2022 12:58:31 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Detaching an Elastic IP address from an EC2 instance

In [8]:
EC2_CLIENT.disassociate_address(AssociationId=response['AssociationId'])

{'ResponseMetadata': {'RequestId': '11b66348-d406-4391-8014-3314b89d600b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '11b66348-d406-4391-8014-3314b89d600b',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '239',
   'date': 'Fri, 04 Feb 2022 13:00:18 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Releasing the Elastic IP address

In [9]:
EC2_CLIENT.release_address(AllocationId=ElasticIP['AllocationId'])

{'ResponseMetadata': {'RequestId': '7b2e9ba2-363d-48b4-8179-3ed7970b0f9e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7b2e9ba2-363d-48b4-8179-3ed7970b0f9e',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '229',
   'date': 'Fri, 04 Feb 2022 13:00:39 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}